# Boosting.Gradient boosting Machine (GBM)(TreeNet)

Разреберем задачу, в которой нужно предсказать доход человека по результатам перепеси населения.
http://archive.ics.uci.edu/ml/datasets/Adult
Описание данных:
http://archive.ics.uci.edu/ml/datasets/adult/adult.names

In [1]:
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt

In [37]:
columns = ['age', 'workclass', 'fnlwgt', 'education', 'education-num',
           'marital-status', 'occupation', 'relationship', 'race', 'sex',
           'capital-gain', 'capital-loss', 'hours-per-week', 'native-country', 'income']
df = pd.read_csv("adult.data",header = None, names = columns, na_values = '?')

In [42]:
df.head(10)

,age,workclass,fnlwgt,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,39,State-gov,77516,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,0
1,50,Self-emp-not-inc,83311,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,0
2,38,Private,215646,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,0
3,53,Private,234721,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,0
4,28,Private,338409,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,0
5,37,Private,284582,14,Married-civ-spouse,Exec-managerial,Wife,White,Female,0,0,40,United-States,0
6,49,Private,160187,5,Married-spouse-absent,Other-service,Not-in-family,Black,Female,0,0,16,Jamaica,0
7,52,Self-emp-not-inc,209642,9,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,45,United-States,1
8,31,Private,45781,14,Never-married,Prof-specialty,Not-in-family,White,Female,14084,0,50,United-States,1
9,42,Private,159449,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,5178,0,40,United-States,1


In [39]:
df = df.drop('education',axis=1)#удаляем колонку т.к уже есть закондированная educ-num

In [40]:
df['income'] = df['income'].map({' <=50K': 0, ' >50K': 1})#кодируем отклик в бинарные значения

In [44]:
df.shape

(32561, 14)

In [43]:
#удаляем строки с NA значениями
df = df.dropna()

In [67]:
test = pd.read_csv("adult.test",header = None, names = columns, na_values = '?',skiprows = 1)
test = test.drop('education',axis=1)
test['income'] = test['income'].map({' <=50K.': 0, ' >50K.': 1})
test = test.dropna()

In [68]:
test.head()

,age,workclass,fnlwgt,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,25,Private,226802,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,0
1,38,Private,89814,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,0
2,28,Local-gov,336951,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,1
3,44,Private,160323,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,1
4,18,?,103497,10,Never-married,?,Own-child,White,Female,0,0,30,United-States,0


Посчитаем распределение классов в отклике

In [46]:
df['income'].value_counts(normalize=True)

0    0.75919
1    0.24081
Name: income, dtype: float64

Разбиваем данные на Обучающую и тестовую выборку. Бинаризуем категориальные признаки (one-hot encoding)

In [69]:
X_train = pd.get_dummies(df).drop('income',axis = 1)
y_train = df['income']

X_test = pd.get_dummies(test).drop('income',axis = 1)
y_test = test['income']


После бинаризации категориальных переменных окажется, что в обучающем множестве содержится на одну колонку больше.
Это от того, что в тестовой выборке нет ни одного представителя  Голландии в колонке native-country

In [70]:
print (len(X_train.columns))
print (len(X_test.columns))

92
91


In [71]:
set(X_train.columns) - set(X_test.columns)
#Разность двух множеств

{'native-country_ Holand-Netherlands'}

Что "полечить" это , создадим полный список всех колонок (из трейна и теста) и переиндексируем колонки в соотсвествии с этим списком. В результате в колонке native-country_ Holand-Netherlands в тесте появится Nan значения, которые мы заменин на  нули.

In [72]:
columns = set(X_train.columns) | set(X_test.columns)
X_train = X_train.reindex(columns = columns).fillna(0)
X_test = X_test.reindex(columns = columns).fillna(0)

Теперь колнки в тесте и трейне доолжны быть идентичными.  В том числе их порядок (колонок). Проверим это all

In [74]:
all(X_train.columns == X_test.columns)

True

Обучение модели:

In [76]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import classification_report

In [77]:
model  = GradientBoostingClassifier(random_state = 42,
                                    n_estimators=100,#Число деревьев участв. в приближении
                                    max_depth=3,#макс. глубина каждого дерева
                                    learning_rate=0.1#параметр уменьш. переобучение, является весом отдельного дерева 
                                   )
model.fit(X_train,y_train)

GradientBoostingClassifier(random_state=42)

In [78]:
y_pred = model.predict(X_test)

In [80]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.89      0.95      0.92     12435
           1       0.79      0.61      0.69      3846

    accuracy                           0.87     16281
   macro avg       0.84      0.78      0.80     16281
weighted avg       0.87      0.87      0.86     16281

